In [253]:
using DelimitedFiles; # Package pour la lecture des donnees
using Random; # Indispensable pour la fonction randperm()

# Fonction de lecture des donnees
# Attention, les valeurs attendues sont des entiers
function readData(file)
data = readdlm(file, ' ', Int, comments = true)
duree_job = data[:,1] # duree d'execution des jobs
fin_job = data[:,2] # date limite de fin des jobs
poids_job = data[:,3] # ponderation des jobs
n = length(duree_job) # Nombre de jobs
return duree_job, fin_job, poids_job, n
end

readData (generic function with 1 method)

In [254]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("P6-simple.dat")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)


********Lecture des donnees**********
nombre de jobs: 10
duree_job: [4, 2, 8, 1, 6, 9, 4, 7, 5, 3]
fin_job: [10, 11, 45, 8, 32, 19, 41, 30, 16, 20]
poids_job: [2, 2, 1, 6, 3, 7, 2, 9, 4, 1]


In [255]:
function evaluation(duree_job, fin_job, poids_job, n, solution)
    cost = 0.0 # complexité = 1
    time = 0.0 # complexité = 1
    jobTmp = 0 ::Int64 # complexité = 1
    for i = 1:n # complexité(boucle) = n*7
        jobTmp = solution[i] # complexité = 1
        time += duree_job[jobTmp] # complexité = 2
        cost += max(0.0, time - fin_job[jobTmp])*poids_job[jobTmp] # complexité = 4
    end
    return cost # complexité = 1
end # complexité(évaluation) = 3 + 7*n + 1 = 7*n + 4

evaluation (generic function with 1 method)

In [256]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
println("\nfirst sol: ", sol, " ")



first sol: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 


In [257]:
cost = evaluation(duree_job, fin_job, poids_job, n, sol)
println("cost: ", cost)


cost: 367.0


In [258]:
function API(solution, n, index)
    newSol = copy(solution)
    if index > 0 && index < n
        tmp = newSol[index]
        newSol[index] = newSol[index+1]
        newSol[index + 1] = tmp
    end
    return newSol
end

API (generic function with 1 method)

In [259]:
"""
La fonction `getFirstImprovementAPI` permet de trouver le premier voisin
ameliorant du voisinage API.

On parcourt tous les index de 1 a n-1 puis on teste la permutation de index
et son successeur immediat index+1. Si on trouve une amelioration de la
solution, on stocke l'index qui a permis l'amelioration, on quitte la boucle.
"""
function getFirstImprovementAPI(duree_job, fin_job, poids_job, n, solution)
# on garde la valeur du cout de la solution courante
    costSol = evaluation(duree_job, fin_job, poids_job, n, solution)
# on gardera une trace de l'index qui a permis l'amelioration
# (si index reste a 0, pas d'amelioration)
index = 0
# on parcourt i entre 1 et n-1 pour faire les echanges entre i et i+1
    for i in 1:n-1
# si la solution apres l'echange API a un plus petit cout
        if evaluation(duree_job, fin_job, poids_job, n, API(solution,n, i)) < costSol
# on stocke l'index qui nous permet d'atteindre cette solution
            index = i
            break # on quitte la boucle qui parcourt les index
        end
    end
    if index != 0 # si on a trouve une meilleure solution,
# la variable index a change
        return API(solution,n, index) # on renvoie la nouvelle solution
        else # sinon
        return solution # on renvoie la solution de dÃ©part
    end
end

getFirstImprovementAPI

In [260]:
function simpleDescent(duree_job, fin_job, poids_job, n, solution)
    bestSol = copy(solution)
    cost = evaluation(duree_job, fin_job, poids_job, n, bestSol)
    bestCost = cost + 1
    while cost < bestCost
        bestCost = cost
        bestSol = getFirstImprovementAPI(duree_job,fin_job,poids_job,n,bestSol)
        cost = evaluation(duree_job, fin_job, poids_job, n, bestSol)
    end
    return bestSol
end

simpleDescent (generic function with 1 method)

In [261]:
sol = simpleDescent(duree_job, fin_job, poids_job, n, sol)
cost = evaluation(duree_job, fin_job, poids_job, n, sol)


41.0

1)

In [262]:
function GPI(solution, n, index1, index2)
    newSol = copy(solution) # complexité = 1
    if index1 > 0 && index1 < n && index2 > 0 && index2 < n # complexité <= 4
        tmp1 = newSol[index1] # complexité = 1
        newSol[index1] = newSol[index2] # complexité = 1
        newSol[index2] = tmp1 # complexité = 1
    end
    return newSol # complexité = 1
end # complexité(GPI) = 1+1+1+4+1+1 = 9

GPI (generic function with 1 method)

2)

In [263]:
function getFirstImprovementGPI(duree_job, fin_job, poids_job, n, solution)
    costSol = evaluation(duree_job, fin_job, poids_job, n, solution) # complexité = 7n + 4
    index1 = index2 = 0 # complexité = 2
    for i1 in 1:n-1, i2 in 2:n # complexité(double boucle) = 7n^3 -7n^2 -14n
        if evaluation(duree_job, fin_job, poids_job, n, GPI(solution,n,i1,i2)) < costSol # complexité <= 7n + 4 + 9 + 1 = 7n + 14
            index1 = i1 # complexité = 1
            index2 = i2 # complexité = 1
            break
        end
    end
    if index1 != 0 # complexité <= 1
        return GPI(solution,n,index1,index2) # complexité = 9
    else
        return solution # complexité = 1
    end
end

getFirstImprovementGPI (generic function with 1 method)

In [264]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
println("\nfirst sol: ", sol, " ")
cost = evaluation(duree_job, fin_job, poids_job, n, sol)
solgpi = getFirstImprovementGPI(duree_job, fin_job, poids_job, n, sol)
costgpi = evaluation(duree_job, fin_job, poids_job, n, solgpi)
println("Cout solution initiale: ", cost, "." , " Cout du premier voisin de la solution initiale par GPI: ", costgpi,".")


first sol: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
Cout solution initiale: 367.0. Cout du premier voisin de la solution initiale par GPI: 335.0.


3) 

complexité(double boucle) = [n*(n-1)] * (complexité évaluation() + complexité GPI + 1)
                          = (n^2 - n)* 7n + 4 + 9 + 1
                          = 7n^3 -7n^2 -14n
                          
complexité totale = 7n + 4 + 2 + 7n^3 -7n^2 -14n + 1 + 9 = 7n^3 -7n^2 -7n +16

4)

In [265]:
fin_job = [10, 11, 45, 8, 32, 19, 41, 30, 16, 20]
job = [1,2,3,4,5,6,7,8,9,10]
couples = [(10,1),(11,2),(45,3),(8,4),(32,5),(19,6),(41,7),(30,8),(16,9),(20,10)]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
println("\nfirst sol: ", sol_new, " ")


first sol: [4, 1, 2, 9, 6, 10, 8, 5, 7, 3] 


La stratégie est de trier les tâches dans l'odre croissant des dates échues dj, de sorte à avoir plus de chance de résoudre les premières tâches en avance.

5)

In [266]:
cost = evaluation(duree_job, fin_job, poids_job, n, sol_new)
println("Cout de la nouvelle solution initiale avec la nouvelle stratégie: ", cost)


Cout de la nouvelle solution initiale avec la nouvelle stratégie: 121.0


6)

In [267]:
function simpleDescent2(duree_job, fin_job, poids_job, n, solution)
    bestSol = copy(solution)
    cost = evaluation(duree_job, fin_job, poids_job, n, bestSol)
    bestCost = cost + 1
    while cost < bestCost
        bestCost = cost
        bestSol = getFirstImprovementGPI(duree_job,fin_job,poids_job,n,bestSol)
        cost = evaluation(duree_job, fin_job, poids_job, n, bestSol)
    end
    return bestSol
end

simpleDescent2 (generic function with 1 method)

In [283]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_1.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [23, 10, 40, 9, 13, 20, 17, 31, 18, 6, 1, 7, 4, 2, 55, 18, 21, 30, 26, 17, 18, 10, 12, 34, 3, 18, 12, 24, 3, 14, 6, 37, 13, 22, 22, 25, 8, 8, 12, 31, 30, 19, 25, 8, 17, 37, 15, 20, 3, 43, 16, 28, 7, 24, 26, 36, 22, 5, 34, 3, 5, 11, 33, 27, 30, 9, 19, 8, 6, 9, 3, 3, 9, 46, 36, 35, 3, 23, 4, 23, 17, 2, 29, 32, 41, 20, 1, 22, 9, 36, 21, 26, 27, 30, 13, 33, 25, 2, 45, 29]
fin_job: [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
poids_job: [4, 8, 2, 8, 6, 7, 6, 10, 8, 7, 3, 9, 5, 2, 3, 9, 3, 6, 10, 8, 8, 9, 4, 10, 10, 1, 7, 4, 1, 1, 10, 9, 6, 8, 7, 1, 10, 8, 8, 4, 2, 9, 4, 9, 

In [285]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 288648.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 251405.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 288406.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 247493.0
Meilleure solution: 247493.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 247493.0

In [286]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_5.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [6, 57, 13, 41, 33, 6, 16, 45, 18, 23, 29, 30, 4, 9, 6, 24, 28, 13, 26, 2, 20, 23, 16, 1, 5, 3, 43, 41, 4, 33, 50, 20, 24, 30, 46, 38, 30, 23, 30, 19, 32, 17, 36, 12, 29, 35, 4, 7, 44, 4, 15, 24, 20, 12, 30, 11, 28, 7, 24, 17, 21, 47, 43, 24, 7, 15, 51, 12, 24, 32, 18, 33, 11, 21, 15, 27, 20, 42, 5, 13, 15, 11, 46, 30, 33, 12, 1, 10, 44, 11, 33, 4, 9, 21, 21, 48, 13, 32, 9, 13]
fin_job: [63, 66, 73, 60, 55, 69, 49, 49, 65, 60, 49, 66, 56, 69, 60, 58, 61, 50, 47, 61, 63, 71, 72, 63, 48, 54, 63, 55, 66, 68, 71, 65, 59, 64, 60, 56, 58, 71, 62, 48, 57, 61, 74, 40, 59, 61, 48, 46, 70, 33, 69, 39, 53, 63, 62, 60, 57, 56, 65, 55, 50, 68, 64, 59, 27, 65, 71, 68, 60, 73, 66, 74, 41, 59, 52, 60, 45, 71, 22, 61, 68, 58, 67, 73, 62, 45, 49, 67, 73, 64, 52, 63, 49, 59, 66, 62, 53, 47, 55, 66]
poids_job: [8, 2, 3, 1, 3, 10, 1, 9, 7, 5, 4, 2, 4, 7, 9, 3, 6, 10, 2, 6, 3, 2, 5, 7, 5, 8, 7, 9, 1, 2, 2, 6, 8, 5, 6, 3, 4, 1, 4, 10, 9, 7,

In [287]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 267389.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 236217.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 273262.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 234371.0
Meilleure solution: 234371.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 234371.0

In [288]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_6.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [22, 30, 17, 16, 39, 10, 4, 15, 16, 4, 39, 24, 8, 14, 36, 9, 21, 16, 7, 8, 17, 16, 39, 40, 1, 29, 1, 11, 32, 4, 15, 20, 7, 26, 3, 20, 22, 51, 46, 5, 45, 46, 26, 9, 17, 37, 29, 44, 22, 39, 35, 1, 19, 5, 45, 6, 16, 24, 1, 2, 8, 28, 9, 9, 6, 23, 33, 34, 15, 7, 14, 55, 12, 34, 13, 1, 11, 23, 10, 16, 37, 10, 34, 12, 6, 10, 36, 1, 35, 11, 7, 35, 8, 35, 3, 47, 24, 7, 13, 18]
fin_job: [63, 54, 47, 67, 69, 64, 62, 46, 62, 48, 61, 73, 61, 56, 60, 54, 59, 53, 62, 62, 64, 60, 70, 51, 50, 50, 56, 69, 58, 62, 54, 64, 9, 67, 48, 72, 71, 69, 63, 49, 62, 72, 63, 59, 40, 67, 65, 64, 66, 56, 68, 72, 62, 75, 72, 53, 50, 54, 50, 34, 63, 60, 71, 75, 44, 61, 71, 71, 58, 62, 70, 68, 33, 77, 47, 67, 52, 65, 55, 35, 60, 61, 58, 70, 60, 60, 60, 61, 58, 26, 45, 62, 44, 61, 51, 68, 54, 51, 49, 68]
poids_job: [7, 5, 1, 2, 5, 7, 4, 5, 1, 9, 10, 3, 4, 9, 8, 2, 2, 10, 9, 9, 4, 6, 10, 1, 1, 9, 5, 3, 7, 5, 5, 9, 3, 7, 10, 4, 10, 2, 9, 10, 1, 10, 1, 3, 

In [289]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 258112.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 285107.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 261199.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 288851.0
Meilleure solution: 258112.0


Meilleure solution: cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 258112.0

In [290]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_7.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [24, 51, 29, 38, 19, 36, 47, 26, 58, 23, 23, 34, 37, 22, 33, 15, 25, 5, 34, 24, 6, 2, 31, 11, 18, 39, 16, 7, 31, 16, 44, 36, 44, 27, 43, 1, 12, 12, 9, 45, 12, 16, 5, 32, 11, 9, 8, 22, 5, 27, 12, 8, 38, 7, 45, 3, 21, 30, 37, 22, 13, 15, 2, 36, 1, 7, 2, 53, 28, 20, 33, 42, 13, 19, 40, 36, 1, 33, 43, 9, 31, 14, 19, 11, 36, 35, 16, 6, 57, 24, 32, 26, 17, 27, 1, 13, 2, 12, 2, 38]
fin_job: [58, 72, 66, 62, 56, 63, 65, 59, 66, 54, 53, 58, 56, 47, 67, 35, 70, 62, 68, 62, 57, 58, 64, 65, 42, 60, 67, 67, 64, 65, 76, 55, 67, 69, 54, 61, 51, 51, 31, 75, 54, 64, 72, 65, 60, 57, 70, 55, 70, 68, 66, 53, 55, 35, 60, 64, 45, 50, 70, 43, 51, 60, 68, 66, 58, 64, 46, 69, 51, 64, 53, 73, 51, 54, 69, 72, 68, 60, 58, 68, 72, 70, 51, 21, 58, 66, 61, 70, 72, 44, 66, 59, 56, 54, 69, 51, 50, 51, 46, 72]
poids_job: [6, 7, 10, 4, 4, 8, 6, 1, 4, 1, 6, 5, 2, 3, 1, 5, 8, 2, 10, 3, 2, 2, 7, 7, 8, 8, 7, 1, 9, 4, 2, 4, 10, 8, 2, 2, 1, 1, 5, 6, 5, 6, 5,

In [291]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 293238.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 249511.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 296186.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 241124.0
Meilleure solution: 241124.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 241124.0

In [292]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_8.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [1, 18, 2, 5, 19, 13, 15, 33, 18, 27, 12, 3, 23, 54, 5, 26, 8, 30, 14, 7, 19, 23, 35, 24, 3, 10, 67, 16, 56, 23, 6, 2, 3, 25, 36, 39, 11, 27, 52, 3, 15, 5, 57, 2, 13, 24, 43, 24, 15, 8, 21, 18, 44, 13, 11, 27, 20, 18, 18, 28, 49, 1, 44, 5, 38, 18, 12, 32, 11, 41, 10, 21, 6, 39, 17, 17, 35, 35, 34, 5, 34, 5, 17, 12, 40, 35, 33, 2, 9, 21, 21, 3, 3, 13, 9, 17, 32, 3, 42, 13]
fin_job: [67, 64, 63, 40, 67, 57, 63, 50, 58, 57, 52, 61, 61, 67, 59, 57, 65, 51, 59, 61, 41, 66, 73, 65, 56, 53, 74, 57, 63, 62, 62, 68, 73, 62, 74, 72, 60, 41, 69, 72, 60, 59, 64, 60, 66, 61, 71, 55, 60, 62, 50, 63, 64, 69, 34, 56, 36, 68, 60, 48, 65, 73, 65, 45, 63, 44, 66, 60, 49, 56, 44, 46, 67, 71, 66, 62, 51, 68, 57, 68, 59, 38, 53, 68, 68, 64, 68, 19, 59, 61, 53, 35, 59, 67, 65, 45, 38, 59, 58, 63]
poids_job: [6, 5, 5, 10, 3, 4, 8, 1, 6, 5, 10, 3, 5, 9, 7, 6, 10, 2, 6, 3, 9, 5, 6, 10, 1, 4, 6, 8, 5, 6, 2, 3, 1, 4, 7, 4, 3, 1, 4, 9, 5, 4, 8, 1

In [293]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 272603.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 321122.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 277344.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 311164.0
Meilleure solution: 272603.0


Meilleure solution: cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 272603.0

In [294]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_9.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [16, 14, 5, 31, 32, 27, 9, 40, 25, 11, 13, 8, 25, 14, 22, 7, 21, 44, 29, 9, 7, 8, 11, 32, 40, 28, 41, 10, 8, 30, 35, 6, 5, 5, 44, 32, 7, 6, 6, 16, 10, 44, 23, 47, 1, 30, 6, 36, 43, 14, 3, 7, 23, 2, 7, 23, 6, 26, 29, 19, 2, 9, 41, 31, 40, 45, 12, 5, 2, 7, 28, 20, 4, 15, 14, 11, 28, 5, 42, 29, 26, 31, 36, 16, 44, 6, 33, 1, 8, 22, 19, 4, 25, 6, 42, 7, 17, 17, 19, 7]
fin_job: [52, 56, 60, 63, 59, 72, 62, 69, 59, 56, 64, 67, 53, 43, 75, 64, 50, 65, 72, 68, 45, 62, 47, 53, 66, 69, 57, 70, 40, 66, 63, 64, 57, 70, 62, 53, 71, 48, 73, 71, 47, 60, 47, 59, 66, 60, 63, 67, 58, 67, 68, 42, 66, 58, 70, 73, 37, 57, 63, 76, 73, 40, 70, 73, 59, 65, 64, 57, 56, 45, 65, 63, 59, 55, 48, 65, 57, 65, 61, 56, 54, 66, 57, 47, 63, 55, 68, 48, 52, 71, 56, 65, 63, 64, 59, 56, 60, 64, 54, 61]
poids_job: [10, 1, 7, 9, 4, 2, 9, 3, 9, 1, 1, 2, 2, 7, 10, 9, 8, 4, 9, 1, 9, 5, 10, 5, 10, 1, 7, 9, 9, 3, 7, 8, 2, 9, 3, 3, 2, 9, 4, 10, 9, 2, 2, 5, 5, 9, 

In [295]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 300335.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 258443.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 296912.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 271286.0
Meilleure solution: 258443.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 258443.0

In [296]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_11.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [4, 17, 12, 16, 4, 27, 2, 30, 8, 9, 7, 1, 13, 6, 14, 32, 32, 42, 8, 7, 19, 46, 24, 26, 6, 29, 2, 37, 6, 33, 15, 16, 9, 10, 23, 31, 2, 26, 34, 15, 12, 42, 39, 6, 8, 16, 18, 44, 47, 15, 5, 15, 10, 12, 6, 20, 8, 1, 4, 35, 4, 3, 51, 12, 46, 40, 27, 27, 48, 36, 31, 19, 6, 10, 20, 39, 7, 6, 17, 26, 14, 35, 11, 25, 23, 3, 1, 30, 14, 9, 24, 43, 4, 26, 24, 13, 8, 10, 4, 22]
fin_job: [62, 57, 51, 47, 70, 71, 43, 60, 69, 55, 60, 41, 31, 61, 35, 59, 63, 65, 37, 58, 65, 71, 63, 61, 59, 62, 42, 72, 67, 67, 64, 61, 50, 72, 59, 56, 61, 64, 50, 61, 57, 61, 76, 52, 67, 52, 70, 67, 70, 60, 49, 57, 41, 66, 60, 55, 71, 48, 57, 67, 64, 49, 74, 51, 68, 59, 51, 61, 62, 55, 63, 52, 51, 43, 53, 68, 59, 55, 48, 71, 59, 68, 68, 66, 67, 62, 61, 48, 65, 39, 68, 64, 58, 57, 61, 44, 67, 33, 60, 56]
poids_job: [2, 1, 9, 1, 9, 8, 8, 4, 6, 9, 2, 9, 4, 1, 6, 7, 2, 1, 6, 1, 8, 6, 6, 8, 1, 9, 8, 7, 8, 7, 5, 2, 10, 6, 4, 1, 5, 6, 4, 2, 10, 5, 6, 8, 9, 7, 8

In [297]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 243734.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 270808.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 244389.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 278532.0
Meilleure solution: 243734.0


Meilleure solution: cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 243734.0

In [298]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_12.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [10, 4, 10, 39, 6, 22, 26, 6, 33, 15, 50, 49, 41, 18, 12, 26, 27, 3, 9, 39, 17, 54, 5, 3, 5, 40, 12, 5, 29, 33, 25, 32, 24, 16, 30, 18, 6, 1, 27, 11, 40, 15, 26, 23, 21, 26, 32, 8, 18, 22, 31, 12, 5, 7, 20, 24, 44, 4, 9, 9, 13, 37, 23, 1, 28, 27, 31, 43, 14, 19, 12, 5, 8, 5, 1, 25, 8, 22, 43, 23, 24, 13, 41, 21, 29, 2, 7, 30, 13, 46, 3, 32, 28, 15, 13, 47, 28, 26, 36, 28]
fin_job: [75, 45, 51, 67, 46, 51, 59, 66, 62, 56, 74, 71, 68, 51, 50, 62, 66, 66, 64, 62, 61, 60, 41, 52, 50, 50, 61, 57, 61, 71, 63, 57, 52, 47, 69, 69, 63, 49, 76, 52, 71, 47, 50, 37, 54, 71, 62, 39, 72, 60, 61, 58, 54, 48, 52, 60, 61, 70, 47, 51, 60, 55, 69, 39, 57, 66, 68, 60, 69, 57, 55, 65, 36, 44, 63, 67, 54, 55, 68, 72, 67, 72, 61, 56, 70, 55, 49, 63, 63, 66, 54, 67, 73, 69, 61, 66, 72, 64, 52, 63]
poids_job: [8, 4, 8, 8, 3, 7, 6, 7, 9, 1, 8, 1, 9, 9, 1, 7, 5, 3, 8, 4, 7, 5, 4, 9, 6, 1, 8, 10, 6, 1, 4, 4, 9, 10, 6, 6, 3, 5, 2, 8, 3, 9, 10, 6,

In [299]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 283890.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 264407.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 283603.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 261984.0
Meilleure solution: 261984.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 261984.0

In [300]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_13.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [19, 19, 30, 24, 15, 16, 7, 11, 12, 21, 3, 17, 15, 3, 7, 29, 6, 37, 15, 28, 26, 2, 17, 41, 25, 43, 3, 20, 7, 40, 36, 2, 17, 37, 10, 36, 16, 17, 46, 35, 2, 11, 29, 3, 3, 24, 13, 4, 19, 5, 28, 21, 15, 18, 13, 24, 39, 60, 28, 50, 26, 29, 45, 4, 24, 8, 52, 24, 4, 30, 16, 38, 38, 12, 28, 12, 28, 2, 53, 5, 12, 13, 5, 19, 9, 27, 36, 26, 24, 35, 11, 5, 2, 17, 15, 21, 2, 39, 39, 35]
fin_job: [66, 71, 49, 63, 45, 73, 53, 70, 63, 62, 50, 64, 51, 65, 60, 59, 68, 72, 44, 59, 66, 54, 40, 65, 69, 58, 43, 62, 59, 67, 61, 66, 50, 59, 60, 67, 60, 38, 69, 60, 57, 53, 49, 66, 65, 59, 43, 51, 63, 64, 66, 63, 55, 56, 69, 50, 64, 67, 50, 62, 69, 60, 65, 60, 54, 60, 75, 52, 51, 60, 74, 61, 64, 59, 63, 69, 64, 60, 62, 58, 64, 60, 68, 62, 56, 40, 66, 46, 74, 64, 63, 57, 62, 41, 65, 47, 57, 66, 63, 70]
poids_job: [6, 3, 9, 10, 3, 7, 10, 10, 5, 7, 4, 7, 10, 5, 6, 7, 10, 8, 4, 4, 10, 10, 3, 3, 4, 8, 3, 4, 5, 5, 5, 1, 10, 3, 6, 9, 7, 5, 8, 6, 7, 8

In [301]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 296473.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 326216.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 300838.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 338078.0
Meilleure solution: 296473.0


Meilleure solution: cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 296473.0

In [302]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_14.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [16, 2, 7, 35, 37, 50, 10, 43, 6, 21, 22, 2, 16, 42, 1, 16, 21, 36, 16, 22, 2, 9, 12, 40, 16, 4, 32, 24, 36, 30, 25, 35, 36, 61, 4, 19, 29, 28, 43, 28, 1, 19, 24, 9, 15, 38, 7, 1, 53, 14, 9, 4, 17, 21, 19, 23, 40, 6, 3, 32, 16, 15, 3, 23, 19, 1, 61, 16, 2, 16, 5, 36, 32, 22, 15, 16, 13, 27, 34, 12, 28, 1, 40, 19, 35, 23, 40, 7, 16, 1, 41, 22, 10, 7, 20, 14, 2, 7, 10, 40]
fin_job: [52, 42, 66, 72, 70, 62, 64, 62, 47, 75, 71, 67, 59, 75, 51, 62, 65, 51, 61, 73, 74, 64, 55, 53, 50, 71, 61, 50, 65, 56, 57, 61, 68, 68, 56, 60, 61, 62, 66, 55, 62, 62, 71, 68, 55, 56, 62, 53, 65, 66, 57, 68, 70, 76, 62, 60, 57, 54, 60, 57, 61, 70, 74, 45, 62, 39, 70, 45, 47, 63, 60, 59, 57, 61, 65, 65, 63, 69, 64, 59, 61, 58, 58, 70, 61, 64, 60, 54, 66, 46, 72, 46, 59, 60, 59, 56, 51, 68, 56, 58]
poids_job: [5, 3, 9, 7, 5, 10, 1, 5, 7, 5, 4, 5, 8, 6, 3, 7, 6, 9, 6, 2, 4, 5, 2, 7, 5, 2, 2, 4, 3, 10, 3, 2, 6, 3, 4, 2, 6, 2, 6, 5, 9, 9, 2, 9, 1

In [303]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 245567.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 210521.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 244652.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 211211.0
Meilleure solution: 210521.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 210521.0

In [304]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_15.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [11, 13, 39, 8, 23, 8, 2, 5, 1, 23, 32, 5, 12, 1, 28, 14, 22, 12, 17, 31, 50, 7, 2, 5, 29, 15, 16, 22, 26, 14, 26, 26, 4, 26, 25, 37, 22, 8, 9, 31, 35, 10, 9, 42, 19, 26, 9, 6, 43, 16, 5, 21, 36, 3, 5, 3, 47, 10, 11, 40, 12, 31, 22, 3, 33, 19, 16, 10, 16, 25, 51, 28, 27, 24, 3, 11, 6, 42, 13, 20, 17, 11, 4, 51, 4, 30, 41, 40, 12, 34, 56, 24, 34, 37, 16, 10, 37, 38, 22, 19]
fin_job: [46, 64, 59, 64, 60, 59, 69, 62, 24, 64, 72, 48, 69, 72, 55, 57, 59, 60, 55, 56, 73, 61, 73, 46, 74, 45, 55, 57, 61, 68, 63, 49, 53, 73, 76, 65, 65, 68, 56, 64, 57, 69, 69, 63, 54, 56, 53, 51, 69, 71, 66, 57, 59, 73, 63, 31, 67, 49, 53, 73, 71, 56, 63, 65, 72, 65, 65, 54, 46, 66, 65, 59, 44, 58, 75, 55, 46, 64, 67, 56, 65, 57, 60, 59, 74, 68, 50, 64, 71, 61, 59, 67, 51, 57, 46, 54, 74, 75, 61, 58]
poids_job: [6, 7, 5, 4, 5, 2, 10, 3, 4, 5, 9, 1, 7, 7, 1, 6, 2, 5, 5, 1, 1, 10, 3, 1, 6, 6, 10, 7, 6, 3, 2, 2, 1, 8, 4, 4, 10, 6, 4, 9, 2, 3, 9, 

In [305]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 269373.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 262167.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 274773.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 259753.0
Meilleure solution: 259753.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2 : 259753.0

In [306]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_16.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [3, 27, 6, 32, 43, 20, 10, 2, 27, 7, 5, 14, 23, 36, 29, 33, 14, 22, 38, 3, 2, 24, 58, 14, 16, 17, 35, 30, 13, 7, 12, 31, 15, 3, 6, 4, 38, 56, 10, 36, 5, 9, 26, 16, 25, 2, 39, 1, 39, 11, 7, 30, 20, 32, 35, 8, 7, 36, 26, 23, 28, 48, 10, 29, 31, 7, 35, 39, 23, 18, 42, 34, 26, 19, 1, 25, 44, 28, 6, 17, 38, 30, 44, 55, 12, 4, 36, 8, 2, 39, 9, 38, 37, 22, 8, 37, 11, 30, 28, 24]
fin_job: [73, 56, 29, 66, 62, 60, 69, 42, 54, 23, 58, 55, 67, 62, 66, 54, 59, 60, 71, 75, 66, 65, 69, 55, 66, 65, 66, 44, 52, 70, 43, 73, 41, 61, 42, 36, 65, 74, 46, 66, 64, 45, 69, 47, 62, 71, 70, 55, 62, 50, 71, 51, 65, 59, 57, 58, 72, 71, 58, 52, 62, 70, 55, 51, 44, 68, 55, 64, 44, 54, 60, 72, 66, 47, 64, 55, 70, 54, 65, 67, 56, 64, 68, 71, 64, 62, 63, 55, 54, 60, 37, 69, 71, 59, 47, 66, 59, 39, 59, 62]
poids_job: [1, 1, 5, 4, 7, 10, 3, 2, 8, 5, 8, 1, 10, 1, 5, 6, 3, 7, 5, 6, 7, 3, 6, 9, 10, 7, 7, 1, 6, 1, 8, 5, 2, 7, 1, 6, 8, 8, 3, 7, 9, 8, 1, 1,

In [307]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 311491.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 248448.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 300736.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 261295.0
Meilleure solution: 248448.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 248448.0

In [308]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_19.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [34, 12, 7, 18, 1, 11, 34, 6, 48, 19, 3, 15, 20, 32, 22, 2, 19, 26, 38, 46, 30, 14, 25, 16, 4, 31, 33, 44, 4, 10, 15, 32, 16, 14, 22, 10, 31, 17, 42, 33, 37, 16, 5, 3, 4, 29, 6, 13, 29, 18, 37, 45, 6, 14, 16, 10, 44, 23, 13, 37, 1, 14, 13, 29, 3, 44, 2, 13, 8, 5, 10, 18, 17, 13, 44, 2, 2, 11, 40, 23, 19, 26, 4, 7, 29, 17, 32, 43, 5, 36, 4, 1, 4, 18, 2, 54, 16, 2, 8, 28]
fin_job: [63, 44, 56, 45, 70, 31, 64, 51, 70, 57, 25, 57, 51, 60, 60, 23, 63, 67, 71, 57, 60, 49, 61, 55, 51, 53, 65, 68, 67, 55, 35, 60, 69, 62, 69, 31, 61, 66, 66, 62, 63, 62, 37, 56, 66, 75, 54, 68, 62, 71, 64, 58, 63, 52, 76, 60, 52, 59, 55, 65, 33, 56, 46, 66, 62, 62, 52, 64, 63, 55, 36, 66, 68, 61, 66, 63, 40, 56, 64, 62, 65, 59, 54, 64, 44, 53, 50, 69, 36, 67, 49, 71, 65, 44, 63, 73, 65, 42, 40, 65]
poids_job: [6, 2, 1, 1, 6, 8, 1, 1, 6, 6, 10, 3, 9, 10, 2, 4, 7, 9, 1, 8, 6, 4, 8, 4, 10, 5, 3, 5, 7, 6, 7, 5, 3, 10, 3, 1, 2, 4, 9, 3, 6, 7, 5, 9, 

In [309]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 241821.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 200809.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 237538.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 201810.0
Meilleure solution: 200809.0


Meilleure solution: cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 200809.0

In [310]:
println("********Lecture des donnees**********")
duree_job, fin_job, poids_job, n = readData("ODD100_20.DAT")
println("nombre de jobs: ",n)
println("duree_job: ", duree_job)
println("fin_job: ", fin_job)
println("poids_job: ", poids_job)

********Lecture des donnees**********
nombre de jobs: 100
duree_job: [30, 46, 19, 22, 44, 46, 1, 6, 5, 43, 44, 8, 3, 22, 28, 41, 20, 16, 22, 43, 12, 12, 24, 10, 15, 32, 9, 17, 35, 50, 14, 47, 28, 36, 33, 33, 23, 14, 7, 29, 10, 18, 42, 40, 5, 5, 37, 17, 17, 51, 38, 39, 11, 16, 26, 11, 19, 30, 17, 1, 2, 11, 7, 7, 51, 36, 28, 5, 20, 20, 31, 12, 9, 18, 9, 12, 30, 3, 8, 10, 20, 35, 20, 16, 29, 19, 14, 11, 1, 15, 6, 16, 16, 27, 44, 36, 34, 19, 21, 37]
fin_job: [62, 69, 49, 49, 58, 68, 68, 56, 72, 60, 59, 58, 64, 54, 66, 65, 60, 62, 52, 61, 64, 51, 75, 45, 46, 46, 67, 38, 62, 72, 66, 75, 52, 54, 70, 73, 63, 73, 39, 70, 70, 66, 70, 57, 38, 63, 68, 70, 60, 65, 59, 59, 72, 58, 52, 62, 55, 74, 49, 36, 62, 71, 56, 55, 72, 70, 39, 66, 60, 58, 62, 48, 57, 53, 63, 37, 74, 74, 71, 51, 64, 74, 73, 73, 69, 63, 74, 64, 61, 66, 73, 62, 72, 60, 60, 58, 66, 63, 64, 64]
poids_job: [4, 4, 8, 8, 3, 1, 9, 1, 9, 3, 8, 2, 5, 6, 2, 3, 9, 1, 2, 6, 7, 4, 8, 7, 5, 5, 4, 5, 2, 6, 10, 9, 5, 4, 8, 10, 5, 10, 8, 9, 6, 4,

In [311]:
sol = collect(1:n) # equivalent a : sol = [i for i = 1:n]
fin_job = [53, 52, 63, 52, 54, 60, 66, 43, 62, 60, 37, 71, 59, 41, 67, 47, 50, 67, 56, 67, 47, 40, 59, 59, 70, 50, 68, 54, 64, 53, 43, 67, 66, 69, 57, 65, 44, 24, 63, 71, 56, 47, 58, 48, 55, 57, 53, 57, 33, 76, 70, 68, 62, 50, 53, 52, 58, 62, 67, 42, 58, 58, 78, 64, 54, 52, 65, 59, 72, 59, 62, 62, 53, 67, 65, 66, 57, 55, 63, 65, 65, 56, 49, 57, 70, 55, 61, 69, 62, 54, 68, 56, 51, 72, 68, 65, 42, 69, 69, 64]
job = [i for i = 1:n]
couples = [(fin_job[j],job[j]) for j = 1:n]
fin_job = sort(fin_job)
sol_new = []
for i = 1:n
    for j = 1:n
        if fin_job[i] == couples[j][1]
            append!(sol_new,couples[j][2])
        end
    end
end
sol_new
sol_new = convert.(Int64,sol_new)
solD = simpleDescent(duree_job, fin_job, poids_job, n, sol)
costD = evaluation(duree_job, fin_job, poids_job, n, solD)
println("cout avec solution initiale sans tri et utilisation de API dans simpleDescent: ", costD)
sol_newD = simpleDescent(duree_job, fin_job, poids_job, n, sol_new)
cost_newD = evaluation(duree_job, fin_job, poids_job, n, sol_newD)
println("cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: ", cost_newD)
solD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol)
costD2 = evaluation(duree_job, fin_job, poids_job, n, solD2)
println("cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: ", costD2)
sol_newD2 = simpleDescent2(duree_job, fin_job, poids_job, n, sol_new)
cost_newD2 = evaluation(duree_job, fin_job, poids_job, n, sol_newD2)
println("cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: ", cost_newD2)
println("Meilleure solution: ", min(costD,cost_newD,costD2,cost_newD2))

cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 347344.0
cout avec solution initiale triée selon les dj et utilisation de API dans simpleDescent: 367239.0
cout avec solution initiale sans tri et utilisation de GPI dans simpleDescent2: 352070.0
cout avec solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2: 379036.0
Meilleure solution: 347344.0


Meilleure solution: cout avec solution initiale sans tri et utilisation de API dans simpleDescent: 347344.0

Les meilleures méthodes sont :
    - solution initiale sans tri et utilisation de API dans simpleDescent
    - solution initiale triée selon les dj et utilisation de GPI dans simpleDescent2
Ces deux méthodes ont été les meilleures 5 fois sur 14.

La troisième meilleure méthode est celle de la solution initiale triée selon les dj et utilisation de API dans simpleDescent, quatre fois meilleure méthode.

Et la moins bonne des méthodes est celle de la solution initiale sans tri et utilisation de GPI dans simpleDescent2 qui n'aura jamais été la meilleure méthode.